# SAE Project : API for Generatives Images

#### Group names : CUISSET Mattéo, VANBANDON Théo, COPIN Lucas, MOUTON Thomas, WAELES Swann, DELCAMBRE Jean-Alexis
#### Antwert special guest : VAN CAMP Loic

## Setup project

In [ ]:
!pip install -r requirement.txt

In [6]:
import torch
import requests
import PIL.Image
from io import BytesIO
import requests
import json
import matplotlib.pyplot as plt
import os

In [3]:
headers = {'Content-Type': 'application/json'}

**To ensure the API works**, please execute the following commands. <br>
This will install the necessary tool for PowerPaint

In [ ]:
os.chdir('./selfhost_stablediffusion_api/generator')
!git lfs clone https://huggingface.co/JunhaoZhuang/PowerPaint_v2/ ./content/checkpoints/
!git clone https://github.com/open-mmlab/PowerPaint.git
os.chdir('../../')

**Launch the API by using the following command in a terminal (make sure you are in this working directory) :**

*python -m selfhost_stablediffusion_api --debug*

If you have some problems with the module "Inpainting", just remove the 'Inpaiting' from the tab "__all__" in the file "selfhost_stablediffusion_api\generator\__init__.py" .<br>
By doing this, features 3 and 5 won't work but the others will work

## Initialize image and mask

By the following, we're gonna use an image and his mask, you can upload it here.
<br> We recommend you to use the PNG format if you want to make sure that the Generative IA will is compatible with you image

In [ ]:
def download_image(url):
    response = requests.get(url)
    return PIL.Image.open(BytesIO(response.content)).convert("RGB")

You can download an image from the web using this function. <br>
Replace the string by the file url in corresponding variables

In [ ]:
init_image = download_image("put_your_url_here.github.com")
mask_image = download_image("put_your_url_here.github.com")

You can use an image on your computer. <br>
Replace the string by the file path in corresponding variables

In [ ]:
init_image = PIL.Image.open("put_your_path_here.png")
mask_image = PIL.Image.open("put_your_path_here.png")

You can still modify these previous variables when you want to change your image.

## Feature \#1 : Generation with Text To Image

### Case 1 : We want to generate a post

We define the constants. <br>
You can change differents parameters in the dictionnary

In [ ]:
data = {
    'prompt': 'Write your prompt here',
    'guidance_scale': 7.5,
    'num_inference_steps' : 50,
    'negative_prompt' : ["an object that you don't wanna see","another"]
}

Launch the query and look at the result image

In [ ]:
url = "http://localhost:8080/txt2img/post"
#If you have some problems with the url, you can replace 'localhost' by '127.0.0.1'

response = requests.post(url, headers=headers, json=data)

try:
    error_message = response.json()
    if 'error' in error_message:
        print(f"Error: {error_message['error']}")
except ValueError:
    if response.status_code != 200:
        print(f"Unknow error, HTTP code : {response.status_code}")
    else:
        image = PIL.Image.open(BytesIO(response.content))
        plt.imshow(image)
        plt.title("Result image")
        plt.axis('off') 
        plt.show()

### Case 2 : We want to generate an avatar

We define the constants. <br>
You can change differents parameters in the dictionnary

#### Make a picture :

In [ ]:
data = {
    'picture' : 'picture',
    'picture_type' : "choose your", #you can choose between "people, animal or background"
    'descriptionPeople' : ["gender","hair_color","hair_length","haircut","nationality","eye_color"], #don't forget to modify the values if you choose 'people', else put an empty list
    'descriptionAnimal' : ["animal","gender","body_color","heigth","environement"], #don't forget to modify the values if you choose 'animal', else put an empty list
    'descriptionBackground' : ["your favorite color","your favorite sport","favorite animal","favorite song","favorite dish","job","hero"] #don't forget to modify the values if you choose 'background', else put an empty list
}

#### Make a paiting :

In [ ]:
data = {
    'picture' : 'paiting',
    'picture_type' : "choose your", #you can choose between "people, animal or background"
    'descriptionPeople' : ["gender","hair_color","hair_length","haircut","nationality","eye_color"], #don't forget to modify the values if you choose 'people', else put an empty list
    'descriptionAnimal' : ["animal","gender","body_color","heigth","environement"], #don't forget to modify the values if you choose 'animal', else put an empty list
    'descriptionBackground' : ["your favorite color","your favorite sport","favorite animal","favorite song","favorite dish","job","hero"], #don't forget to modify the values if you choose 'background', else put an empty list
    'artist' : "an artist like Pablo Picasso"
}

#### The result

Launch the query and look at the result image

In [ ]:
url = "http://localhost:8080/txt2img/avatar"
#If you have some problems with the url, you can replace 'localhost' by '127.0.0.1'

response = requests.post(url, headers=headers, json=data)

try:
    error_message = response.json()
    if 'error' in error_message:
        print(f"Error: {error_message['error']}")
except ValueError:
    if response.status_code != 200:
        print(f"Unknow error, HTTP code : {response.status_code}")
    else:
        image = PIL.Image.open(BytesIO(response.content))
        plt.imshow(image)
        plt.title("Result image")
        plt.axis('off') 
        plt.show()

## Feature \#2 : Generation with Image To Image

We define the constants. <br>
You can change differents parameters in the dictionnary

In [ ]:
data = {
    'prompt': 'Write your prompt here',
    'strength': 0.75,
    'num_inference_steps' : 50,
    'image' : init_image
}

Launch the query and look at the result image

In [ ]:
url = "http://localhost:8080/img2img"
#If you have some problems with the url, you can replace 'localhost' by '127.0.0.1'

response = requests.post(url, headers=headers, json=data)

try:
    error_message = response.json()
    if 'error' in error_message:
        print(f"Error: {error_message['error']}")
except ValueError:
    if response.status_code != 200:
        print(f"Unknow error, HTTP code : {response.status_code}")
    else:
        image = PIL.Image.open(BytesIO(response.content))
        plt.imshow(image)
        plt.title("Result image")
        plt.axis('off') 
        plt.show()

## Feature \#3 : Generation with Inpaiting

We define the constants. <br>
You can change differents parameters in the dictionnary

#### By provide a mask

In [ ]:
data = {
    'strategy' : 'Choose your strategy', #You can choose the strategy, type 1 for "Local editing", 2 for "Object removal" and 3 for "Object additionnal"
    'prompt': 'Write your prompt here',
    'image' : init_image,
    'mask' : mask_image
}

#### Without provide a mask

The mask will be generated automatically

In [ ]:
data = {
    'strategy' : 'Choose your strategy', #You can choose the strategy, type 1 for "Local editing", 2 for "Object removal" and 3 for "Object additionnal"
    'prompt': 'Write your prompt here',
    'image' : init_image
}

#### The result

Launch the query and look at the result image

In [ ]:
url = "http://localhost:8080/impaiting"
#If you have some problems with the url, you can replace 'localhost' by '127.0.0.1'

response = requests.post(url, headers=headers, json=data)

try:
    error_message = response.json()
    if 'error' in error_message:
        print(f"Error: {error_message['error']}")
except ValueError:
    if response.status_code != 200:
        print(f"Unknow error, HTTP code : {response.status_code}")
    else:
        image = PIL.Image.open(BytesIO(response.content))
        plt.imshow(image)
        plt.title("Result image")
        plt.axis('off') 
        plt.show()

## Feature \#4 : Text To Image with FineTuning

This part is currently not implemented in our API, sorry for the inconvenience but you can find all methods if you want to test them.

## Feature \#5 : Text To Inpaiting with FineTuning

We define the constants. <br>
You can change differents parameters in the dictionnary

#### By provide a mask

In [ ]:
data = {
    'train_prompt_instance' : "Write your training prompt here",
    'class_prompt' : 'Describe what is in the training images',
    'input_image' : init_image,
    'user_prompt' : "Write your prompt here, this one is for the result image",
    'strength' : 0.6,
    'training_images' : ["an image to train the model","another"], #You can use PIL.Image.open to open your images, like we did for the init_image and mask_image
    'user_mask' : mask_image
}

#### Without provide a mask

In [ ]:
data = {
    'train_prompt_instance' : "Write your training prompt here",
    'class_prompt' : 'Describe what is in the training images',
    'input_image' : init_image,
    'user_prompt' : "Write your prompt here, this one is for the result image",
    'strength' : 0.6,
    'training_images' : ["an image to train the model","another"], #You can use PIL.Image.open to open your images, like we did for the init_image and mask_image
}

#### The result

Launch the query and look at the result image

In [ ]:
url = "http://localhost:8080/fine-inpaiting"
#If you have some problems with the url, you can replace 'localhost' by '127.0.0.1'

response = requests.post(url, headers=headers, json=data)

try:
    error_message = response.json()
    if 'error' in error_message:
        print(f"Error: {error_message['error']}")
except ValueError:
    if response.status_code != 200:
        print(f"Unknow error, HTTP code : {response.status_code}")
    else:
        image = PIL.Image.open(BytesIO(response.content))
        plt.imshow(image)
        plt.title("Result image")
        plt.axis('off') 
        plt.show()